In [1]:
import xarray as xr
import glob
import os
from sublimpy import utils
import numpy as np 
import pandas as pd
pd.set_option("display.max_rows",300)
import altair as alt

In [2]:
filepath = "/Users/elischwat/Development/data/sublimationofsnow/sosqc/sos_qc_geo_tiltcor_v20240307/"
files = glob.glob(os.path.join(filepath, "*.nc"))

In [3]:
# sort files and remove datasets before November
files = sorted(files)[3:] # should be [3:]

# Combine daily datasets into a season-long 5min dataset 

Include only eddy covariance latent heat flux measurements

In [4]:
vars_i_want_1 = [
        'w_h2o__3m_c',    'w_tc__3m_c',  'w_3m_c',   'h2o_3m_c',  'tc_3m_c',  'irgadiag_3m_c',  'ldiag_3m_c',  'counts_3m_c',  'counts_3m_c_1',  'counts_3m_c_2',   
        'w_h2o__5m_c',    'w_tc__5m_c',  'w_5m_c',   'h2o_5m_c',  'tc_5m_c',  'irgadiag_5m_c',  'ldiag_5m_c',  'counts_5m_c',  'counts_5m_c_1',  'counts_5m_c_2',   
        'w_h2o__10m_c',   'w_tc__10m_c', 'w_10m_c',  'h2o_10m_c', 'tc_10m_c', 'irgadiag_10m_c', 'ldiag_10m_c', 'counts_10m_c', 'counts_10m_c_1', 'counts_10m_c_2',  
        'w_h2o__15m_c',   'w_tc__15m_c', 'w_15m_c',  'h2o_15m_c', 'tc_15m_c', 'irgadiag_15m_c', 'ldiag_15m_c', 'counts_15m_c', 'counts_15m_c_1', 'counts_15m_c_2',  
        'w_h2o__20m_c',   'w_tc__20m_c', 'w_20m_c',  'h2o_20m_c', 'tc_20m_c', 'irgadiag_20m_c', 'ldiag_20m_c', 'counts_20m_c', 'counts_20m_c_1', 'counts_20m_c_2',  
        'w_h2o__3m_ue',   'w_tc__3m_ue', 'w_3m_ue',  'h2o_3m_ue', 'tc_3m_ue', 'irgadiag_3m_ue', 'ldiag_3m_ue', 'counts_3m_ue', 'counts_3m_ue_1', 'counts_3m_ue_2',  
        'w_h2o__10m_ue',  'w_tc__10m_ue','w_10m_ue', 'h2o_10m_ue','tc_10m_ue','irgadiag_10m_ue','ldiag_10m_ue','counts_10m_ue','counts_10m_ue_1','counts_10m_ue_2', 
        'w_h2o__3m_uw',   'w_tc__3m_uw', 'w_3m_uw',  'h2o_3m_uw', 'tc_3m_uw', 'irgadiag_3m_uw', 'ldiag_3m_uw', 'counts_3m_uw', 'counts_3m_uw_1', 'counts_3m_uw_2',  
        'w_h2o__10m_uw',  'w_tc__10m_uw','w_10m_uw', 'h2o_10m_uw','tc_10m_uw','irgadiag_10m_uw','ldiag_10m_uw','counts_10m_uw','counts_10m_uw_1','counts_10m_uw_2', 
        'w_h2o__3m_d',    'w_tc__3m_d',  'w_3m_d',   'h2o_3m_d',  'tc_3m_d',  'irgadiag_3m_d',  'ldiag_3m_d',  'counts_3m_d',  'counts_3m_d_1',  'counts_3m_d_2',   
        'w_h2o__10m_d',   'w_tc__10m_d', 'w_10m_d',  'h2o_10m_d', 'tc_10m_d', 'irgadiag_10m_d', 'ldiag_10m_d', 'counts_10m_d', 'counts_10m_d_1', 'counts_10m_d_2',  
    ]
vars_i_want_2 = [
    'u_u_u__3m_c',      'v_v_v__3m_c',          'w_w_w__3m_c',      'tc_tc_tc__3m_c',
    'u_u_u__5m_c',      'v_v_v__5m_c',          'w_w_w__5m_c',      'tc_tc_tc__5m_c',
    'u_u_u__10m_c',     'v_v_v__10m_c',         'w_w_w__10m_c',     'tc_tc_tc__10m_c',
    'u_u_u__15m_c',     'v_v_v__15m_c',         'w_w_w__15m_c',     'tc_tc_tc__15m_c',
    'u_u_u__20m_c',     'v_v_v__20m_c',         'w_w_w__20m_c',     'tc_tc_tc__20m_c',
    'u_u_u__3m_ue',     'v_v_v__3m_ue',         'w_w_w__3m_ue',     'tc_tc_tc__3m_ue',
    'u_u_u__10m_ue',    'v_v_v__10m_ue',        'w_w_w__10m_ue',    'tc_tc_tc__10m_ue',    
    'u_u_u__3m_uw',     'v_v_v__3m_uw',         'w_w_w__3m_uw',     'tc_tc_tc__3m_uw',
    'u_u_u__10m_uw',    'v_v_v__10m_uw',        'w_w_w__10m_uw',    'tc_tc_tc__10m_uw',    
    'u_u_u__3m_d',      'v_v_v__3m_d',          'w_w_w__3m_d',      'tc_tc_tc__3m_d',
    'u_u_u__10m_d',     'v_v_v__10m_d',         'w_w_w__10m_d',     'tc_tc_tc__10m_d',
]
all_vars_i_want = vars_i_want_1 + vars_i_want_2
datasets = []
for fn in files:
    ds = xr.open_dataset(fn)
    vars_to_grab = []
    for var in all_vars_i_want:
        if var in ds:
            vars_to_grab.append(var)
    ds = ds[vars_to_grab]
    datasets.append(ds)

In [5]:
sos_ds = xr.concat(datasets, dim='time')

In [6]:
sos_ds = utils.fill_missing_timestamps(sos_ds)

In [7]:
print('w_h2o__3m_c',    np.isnan(sos_ds['w_h2o__3m_c']).sum().item())
print('w_h2o__5m_c',    np.isnan(sos_ds['w_h2o__5m_c']).sum().item())
print('w_h2o__10m_c',   np.isnan(sos_ds['w_h2o__10m_c']).sum().item())
print('w_h2o__15m_c',   np.isnan(sos_ds['w_h2o__15m_c']).sum().item())
print('w_h2o__20m_c',   np.isnan(sos_ds['w_h2o__20m_c']).sum().item())
print('w_h2o__3m_ue',   np.isnan(sos_ds['w_h2o__3m_ue']).sum().item())
print('w_h2o__10m_ue',  np.isnan(sos_ds['w_h2o__10m_ue']).sum().item())
print('w_h2o__3m_uw',   np.isnan(sos_ds['w_h2o__3m_uw']).sum().item())
print('w_h2o__10m_uw',  np.isnan(sos_ds['w_h2o__10m_uw']).sum().item())
print('w_h2o__3m_d',    np.isnan(sos_ds['w_h2o__3m_d']).sum().item())
print('w_h2o__10m_d',   np.isnan(sos_ds['w_h2o__10m_d']).sum().item())

w_h2o__3m_c 4575
w_h2o__5m_c 6424
w_h2o__10m_c 3486
w_h2o__15m_c 1467
w_h2o__20m_c 5257
w_h2o__3m_ue 6654
w_h2o__10m_ue 5583
w_h2o__3m_uw 6274
w_h2o__10m_uw 7646
w_h2o__3m_d 6816
w_h2o__10m_d 6157


# Assign nans when the irgaflags are non-zero

In [8]:
# assign nans to lh flux measurements when theres a non-zero irga flag
sos_ds['w_h2o__3m_c'] =     sos_ds['w_h2o__3m_c'].where(sos_ds['irgadiag_3m_c'] == 0)
sos_ds['w_h2o__5m_c'] =     sos_ds['w_h2o__5m_c'].where(sos_ds['irgadiag_5m_c'] == 0)
sos_ds['w_h2o__10m_c'] =    sos_ds['w_h2o__10m_c'].where(sos_ds['irgadiag_10m_c'] == 0)
sos_ds['w_h2o__15m_c'] =    sos_ds['w_h2o__15m_c'].where(sos_ds['irgadiag_15m_c'] == 0)
sos_ds['w_h2o__20m_c'] =    sos_ds['w_h2o__20m_c'].where(sos_ds['irgadiag_20m_c'] == 0)
sos_ds['w_h2o__3m_ue'] =    sos_ds['w_h2o__3m_ue'].where(sos_ds['irgadiag_3m_ue'] == 0)
sos_ds['w_h2o__10m_ue'] =   sos_ds['w_h2o__10m_ue'].where(sos_ds['irgadiag_10m_ue'] == 0)
sos_ds['w_h2o__3m_uw'] =    sos_ds['w_h2o__3m_uw'].where(sos_ds['irgadiag_3m_uw'] == 0)
sos_ds['w_h2o__10m_uw'] =   sos_ds['w_h2o__10m_uw'].where(sos_ds['irgadiag_10m_uw'] == 0)
sos_ds['w_h2o__3m_d'] =     sos_ds['w_h2o__3m_d'].where(sos_ds['irgadiag_3m_d'] == 0)
sos_ds['w_h2o__10m_d'] =    sos_ds['w_h2o__10m_d'].where(sos_ds['irgadiag_10m_d'] == 0)

# assign nans to h2o measurements when theres a non-zero irga flag
sos_ds['h2o_3m_c'] =     sos_ds['h2o_3m_c'].where(sos_ds['irgadiag_3m_c'] == 0)
sos_ds['h2o_5m_c'] =     sos_ds['h2o_5m_c'].where(sos_ds['irgadiag_5m_c'] == 0)
sos_ds['h2o_10m_c'] =    sos_ds['h2o_10m_c'].where(sos_ds['irgadiag_10m_c'] == 0)
sos_ds['h2o_15m_c'] =    sos_ds['h2o_15m_c'].where(sos_ds['irgadiag_15m_c'] == 0)
sos_ds['h2o_20m_c'] =    sos_ds['h2o_20m_c'].where(sos_ds['irgadiag_20m_c'] == 0)
sos_ds['h2o_3m_ue'] =    sos_ds['h2o_3m_ue'].where(sos_ds['irgadiag_3m_ue'] == 0)
sos_ds['h2o_10m_ue'] =   sos_ds['h2o_10m_ue'].where(sos_ds['irgadiag_10m_ue'] == 0)
sos_ds['h2o_3m_uw'] =    sos_ds['h2o_3m_uw'].where(sos_ds['irgadiag_3m_uw'] == 0)
sos_ds['h2o_10m_uw'] =   sos_ds['h2o_10m_uw'].where(sos_ds['irgadiag_10m_uw'] == 0)
sos_ds['h2o_3m_d'] =     sos_ds['h2o_3m_d'].where(sos_ds['irgadiag_3m_d'] == 0)
sos_ds['h2o_10m_d'] =    sos_ds['h2o_10m_d'].where(sos_ds['irgadiag_10m_d'] == 0)

In [9]:
print('w_h2o__3m_c',    np.isnan(sos_ds['w_h2o__3m_c']).sum().item())
print('w_h2o__5m_c',    np.isnan(sos_ds['w_h2o__5m_c']).sum().item())
print('w_h2o__10m_c',   np.isnan(sos_ds['w_h2o__10m_c']).sum().item())
print('w_h2o__15m_c',   np.isnan(sos_ds['w_h2o__15m_c']).sum().item())
print('w_h2o__20m_c',   np.isnan(sos_ds['w_h2o__20m_c']).sum().item())
print('w_h2o__3m_ue',   np.isnan(sos_ds['w_h2o__3m_ue']).sum().item())
print('w_h2o__10m_ue',  np.isnan(sos_ds['w_h2o__10m_ue']).sum().item())
print('w_h2o__3m_uw',   np.isnan(sos_ds['w_h2o__3m_uw']).sum().item())
print('w_h2o__10m_uw',  np.isnan(sos_ds['w_h2o__10m_uw']).sum().item())
print('w_h2o__3m_d',    np.isnan(sos_ds['w_h2o__3m_d']).sum().item())
print('w_h2o__10m_d',   np.isnan(sos_ds['w_h2o__10m_d']).sum().item())

w_h2o__3m_c 7444
w_h2o__5m_c 8593
w_h2o__10m_c 6260
w_h2o__15m_c 10735
w_h2o__20m_c 11917
w_h2o__3m_ue 7767
w_h2o__10m_ue 9704
w_h2o__3m_uw 7403
w_h2o__10m_uw 10310
w_h2o__3m_d 9461
w_h2o__10m_d 8716


# Assign nans when skewness of temperature and wind components is outside range [-2,2]

In [10]:
# def skewness_is_outside_range(suffix = '3m_c'):
#     u_skewness_flag = np.abs(sos_ds[f"u_u_u__{suffix}"]) > 2
#     v_skewness_flag = np.abs(sos_ds[f"v_v_v__{suffix}"]) > 2
#     w_skewness_flag = np.abs(sos_ds[f"w_w_w__{suffix}"]) > 2
#     tc_skewness_flag = np.abs(sos_ds[f"tc_tc_tc__{suffix}"]) > 2
#     return np.logical_or(
#         np.logical_or(
#             np.logical_or(
#                 u_skewness_flag,
#                 v_skewness_flag
#             ),
#             w_skewness_flag
#         ),
#         tc_skewness_flag
#     )

# sos_ds['w_h2o__3m_c'].loc[skewness_is_outside_range('3m_c')] = np.nan
# sos_ds['w_h2o__5m_c'].loc[skewness_is_outside_range('5m_c')] = np.nan
# sos_ds['w_h2o__10m_c'].loc[skewness_is_outside_range('10m_c')] = np.nan
# sos_ds['w_h2o__15m_c'].loc[skewness_is_outside_range('15m_c')] = np.nan
# sos_ds['w_h2o__20m_c'].loc[skewness_is_outside_range('20m_c')] = np.nan
# sos_ds['w_h2o__3m_ue'].loc[skewness_is_outside_range('3m_ue')] = np.nan
# sos_ds['w_h2o__10m_ue'].loc[skewness_is_outside_range('10m_ue')] = np.nan
# sos_ds['w_h2o__3m_uw'].loc[skewness_is_outside_range('3m_uw')] = np.nan
# sos_ds['w_h2o__10m_uw'].loc[skewness_is_outside_range('10m_uw')] = np.nan
# sos_ds['w_h2o__3m_d'].loc[skewness_is_outside_range('3m_d')] = np.nan
# sos_ds['w_h2o__10m_d'].loc[skewness_is_outside_range('10m_d')] = np.nan

# sos_ds['w_tc__3m_c'].loc[skewness_is_outside_range('3m_c')] = np.nan
# sos_ds['w_tc__5m_c'].loc[skewness_is_outside_range('5m_c')] = np.nan
# sos_ds['w_tc__10m_c'].loc[skewness_is_outside_range('10m_c')] = np.nan
# sos_ds['w_tc__15m_c'].loc[skewness_is_outside_range('15m_c')] = np.nan
# sos_ds['w_tc__20m_c'].loc[skewness_is_outside_range('20m_c')] = np.nan
# sos_ds['w_tc__3m_ue'].loc[skewness_is_outside_range('3m_ue')] = np.nan
# sos_ds['w_tc__10m_ue'].loc[skewness_is_outside_range('10m_ue')] = np.nan
# sos_ds['w_tc__3m_uw'].loc[skewness_is_outside_range('3m_uw')] = np.nan
# sos_ds['w_tc__10m_uw'].loc[skewness_is_outside_range('10m_uw')] = np.nan
# sos_ds['w_tc__3m_d'].loc[skewness_is_outside_range('3m_d')] = np.nan
# sos_ds['w_tc__10m_d'].loc[skewness_is_outside_range('10m_d')] = np.nan

In [11]:
# print('w_h2o__3m_c',    np.isnan(sos_ds['w_h2o__3m_c']).sum().item())
# print('w_h2o__5m_c',    np.isnan(sos_ds['w_h2o__5m_c']).sum().item())
# print('w_h2o__10m_c',   np.isnan(sos_ds['w_h2o__10m_c']).sum().item())
# print('w_h2o__15m_c',   np.isnan(sos_ds['w_h2o__15m_c']).sum().item())
# print('w_h2o__20m_c',   np.isnan(sos_ds['w_h2o__20m_c']).sum().item())
# print('w_h2o__3m_ue',   np.isnan(sos_ds['w_h2o__3m_ue']).sum().item())
# print('w_h2o__10m_ue',  np.isnan(sos_ds['w_h2o__10m_ue']).sum().item())
# print('w_h2o__3m_uw',   np.isnan(sos_ds['w_h2o__3m_uw']).sum().item())
# print('w_h2o__10m_uw',  np.isnan(sos_ds['w_h2o__10m_uw']).sum().item())
# print('w_h2o__3m_d',    np.isnan(sos_ds['w_h2o__3m_d']).sum().item())
# print('w_h2o__10m_d',   np.isnan(sos_ds['w_h2o__10m_d']).sum().item())

# Assign nans when ldiag > 0.01 (NCAR recommended, from ISFS_netCDF_FileConventions.pdf)

"The average of ldiag is then the fraction of time over each average period that
diagbits was non-zero."

In [12]:
# sos_ds['w_h2o__3m_c'] =     sos_ds['w_h2o__3m_c'].where(sos_ds['ldiag_3m_c'] <= 0.01)
# sos_ds['w_h2o__5m_c'] =     sos_ds['w_h2o__5m_c'].where(sos_ds['ldiag_5m_c'] <= 0.01)
# sos_ds['w_h2o__10m_c'] =    sos_ds['w_h2o__10m_c'].where(sos_ds['ldiag_10m_c'] <= 0.01)
# sos_ds['w_h2o__15m_c'] =    sos_ds['w_h2o__15m_c'].where(sos_ds['ldiag_15m_c'] <= 0.01)
# sos_ds['w_h2o__20m_c'] =    sos_ds['w_h2o__20m_c'].where(sos_ds['ldiag_20m_c'] <= 0.01)
# sos_ds['w_h2o__3m_ue'] =    sos_ds['w_h2o__3m_ue'].where(sos_ds['ldiag_3m_ue'] <= 0.01)
# sos_ds['w_h2o__10m_ue'] =   sos_ds['w_h2o__10m_ue'].where(sos_ds['ldiag_10m_ue'] <= 0.01)
# sos_ds['w_h2o__3m_uw'] =    sos_ds['w_h2o__3m_uw'].where(sos_ds['ldiag_3m_uw'] <= 0.01)
# sos_ds['w_h2o__10m_uw'] =   sos_ds['w_h2o__10m_uw'].where(sos_ds['ldiag_10m_uw'] <= 0.01)
# sos_ds['w_h2o__3m_d'] =     sos_ds['w_h2o__3m_d'].where(sos_ds['ldiag_3m_d'] <= 0.01)
# sos_ds['w_h2o__10m_d'] =    sos_ds['w_h2o__10m_d'].where(sos_ds['ldiag_10m_d'] <= 0.01)

# sos_ds['w_tc__3m_c'] =     sos_ds['w_tc__3m_c'].where(sos_ds['ldiag_3m_c'] <= 0.01)
# sos_ds['w_tc__5m_c'] =     sos_ds['w_tc__5m_c'].where(sos_ds['ldiag_5m_c'] <= 0.01)
# sos_ds['w_tc__10m_c'] =    sos_ds['w_tc__10m_c'].where(sos_ds['ldiag_10m_c'] <= 0.01)
# sos_ds['w_tc__15m_c'] =    sos_ds['w_tc__15m_c'].where(sos_ds['ldiag_15m_c'] <= 0.01)
# sos_ds['w_tc__20m_c'] =    sos_ds['w_tc__20m_c'].where(sos_ds['ldiag_20m_c'] <= 0.01)
# sos_ds['w_tc__3m_ue'] =    sos_ds['w_tc__3m_ue'].where(sos_ds['ldiag_3m_ue'] <= 0.01)
# sos_ds['w_tc__10m_ue'] =   sos_ds['w_tc__10m_ue'].where(sos_ds['ldiag_10m_ue'] <= 0.01)
# sos_ds['w_tc__3m_uw'] =    sos_ds['w_tc__3m_uw'].where(sos_ds['ldiag_3m_uw'] <= 0.01)
# sos_ds['w_tc__10m_uw'] =   sos_ds['w_tc__10m_uw'].where(sos_ds['ldiag_10m_uw'] <= 0.01)
# sos_ds['w_tc__3m_d'] =     sos_ds['w_tc__3m_d'].where(sos_ds['ldiag_3m_d'] <= 0.01)
# sos_ds['w_tc__10m_d'] =    sos_ds['w_tc__10m_d'].where(sos_ds['ldiag_10m_d'] <= 0.01)

# sos_ds['tc_3m_c'] =     sos_ds['tc_3m_c'].where(sos_ds['ldiag_3m_c'] <= 0.01)
# sos_ds['tc_5m_c'] =     sos_ds['tc_5m_c'].where(sos_ds['ldiag_5m_c'] <= 0.01)
# sos_ds['tc_10m_c'] =    sos_ds['tc_10m_c'].where(sos_ds['ldiag_10m_c'] <= 0.01)
# sos_ds['tc_15m_c'] =    sos_ds['tc_15m_c'].where(sos_ds['ldiag_15m_c'] <= 0.01)
# sos_ds['tc_20m_c'] =    sos_ds['tc_20m_c'].where(sos_ds['ldiag_20m_c'] <= 0.01)
# sos_ds['tc_3m_ue'] =    sos_ds['tc_3m_ue'].where(sos_ds['ldiag_3m_ue'] <= 0.01)
# sos_ds['tc_10m_ue'] =   sos_ds['tc_10m_ue'].where(sos_ds['ldiag_10m_ue'] <= 0.01)
# sos_ds['tc_3m_uw'] =    sos_ds['tc_3m_uw'].where(sos_ds['ldiag_3m_uw'] <= 0.01)
# sos_ds['tc_10m_uw'] =   sos_ds['tc_10m_uw'].where(sos_ds['ldiag_10m_uw'] <= 0.01)
# sos_ds['tc_3m_d'] =     sos_ds['tc_3m_d'].where(sos_ds['ldiag_3m_d'] <= 0.01)
# sos_ds['tc_10m_d'] =    sos_ds['tc_10m_d'].where(sos_ds['ldiag_10m_d'] <= 0.01)

# sos_ds['w_3m_c'] =     sos_ds['w_3m_c'].where(sos_ds['ldiag_3m_c'] <= 0.01)
# sos_ds['w_5m_c'] =     sos_ds['w_5m_c'].where(sos_ds['ldiag_5m_c'] <= 0.01)
# sos_ds['w_10m_c'] =    sos_ds['w_10m_c'].where(sos_ds['ldiag_10m_c'] <= 0.01)
# sos_ds['w_15m_c'] =    sos_ds['w_15m_c'].where(sos_ds['ldiag_15m_c'] <= 0.01)
# sos_ds['w_20m_c'] =    sos_ds['w_20m_c'].where(sos_ds['ldiag_20m_c'] <= 0.01)
# sos_ds['w_3m_ue'] =    sos_ds['w_3m_ue'].where(sos_ds['ldiag_3m_ue'] <= 0.01)
# sos_ds['w_10m_ue'] =   sos_ds['w_10m_ue'].where(sos_ds['ldiag_10m_ue'] <= 0.01)
# sos_ds['w_3m_uw'] =    sos_ds['w_3m_uw'].where(sos_ds['ldiag_3m_uw'] <= 0.01)
# sos_ds['w_10m_uw'] =   sos_ds['w_10m_uw'].where(sos_ds['ldiag_10m_uw'] <= 0.01)
# sos_ds['w_3m_d'] =     sos_ds['w_3m_d'].where(sos_ds['ldiag_3m_d'] <= 0.01)
# sos_ds['w_10m_d'] =    sos_ds['w_10m_d'].where(sos_ds['ldiag_10m_d'] <= 0.01)

# # sos_ds['u_3m_c'] =     sos_ds['u_3m_c'].where(sos_ds['ldiag_3m_c'] <= 0.01)
# # sos_ds['u_5m_c'] =     sos_ds['u_5m_c'].where(sos_ds['ldiag_5m_c'] <= 0.01)
# # sos_ds['u_10m_c'] =    sos_ds['u_10m_c'].where(sos_ds['ldiag_10m_c'] <= 0.01)
# # sos_ds['u_15m_c'] =    sos_ds['u_15m_c'].where(sos_ds['ldiag_15m_c'] <= 0.01)
# # sos_ds['u_20m_c'] =    sos_ds['u_20m_c'].where(sos_ds['ldiag_20m_c'] <= 0.01)
# # sos_ds['u_3m_ue'] =    sos_ds['u_3m_ue'].where(sos_ds['ldiag_3m_ue'] <= 0.01)
# # sos_ds['u_10m_ue'] =   sos_ds['u_10m_ue'].where(sos_ds['ldiag_10m_ue'] <= 0.01)
# # sos_ds['u_3m_uw'] =    sos_ds['u_3m_uw'].where(sos_ds['ldiag_3m_uw'] <= 0.01)
# # sos_ds['u_10m_uw'] =   sos_ds['u_10m_uw'].where(sos_ds['ldiag_10m_uw'] <= 0.01)
# # sos_ds['u_3m_d'] =     sos_ds['u_3m_d'].where(sos_ds['ldiag_3m_d'] <= 0.01)
# # sos_ds['u_10m_d'] =    sos_ds['u_10m_d'].where(sos_ds['ldiag_10m_d'] <= 0.01)

# # sos_ds['v_3m_c'] =     sos_ds['v_3m_c'].where(sos_ds['ldiag_3m_c'] <= 0.01)
# # sos_ds['v_5m_c'] =     sos_ds['v_5m_c'].where(sos_ds['ldiag_5m_c'] <= 0.01)
# # sos_ds['v_10m_c'] =    sos_ds['v_10m_c'].where(sos_ds['ldiag_10m_c'] <= 0.01)
# # sos_ds['v_15m_c'] =    sos_ds['v_15m_c'].where(sos_ds['ldiag_15m_c'] <= 0.01)
# # sos_ds['v_20m_c'] =    sos_ds['v_20m_c'].where(sos_ds['ldiag_20m_c'] <= 0.01)
# # sos_ds['v_3m_ue'] =    sos_ds['v_3m_ue'].where(sos_ds['ldiag_3m_ue'] <= 0.01)
# # sos_ds['v_10m_ue'] =   sos_ds['v_10m_ue'].where(sos_ds['ldiag_10m_ue'] <= 0.01)
# # sos_ds['v_3m_uw'] =    sos_ds['v_3m_uw'].where(sos_ds['ldiag_3m_uw'] <= 0.01)
# # sos_ds['v_10m_uw'] =   sos_ds['v_10m_uw'].where(sos_ds['ldiag_10m_uw'] <= 0.01)
# # sos_ds['v_3m_d'] =     sos_ds['v_3m_d'].where(sos_ds['ldiag_3m_d'] <= 0.01)
# # sos_ds['v_10m_d'] =    sos_ds['v_10m_d'].where(sos_ds['ldiag_10m_d'] <= 0.01)

In [13]:
print('w_h2o__3m_c',    np.isnan(sos_ds['w_h2o__3m_c']).sum().item())
print('w_h2o__5m_c',    np.isnan(sos_ds['w_h2o__5m_c']).sum().item())
print('w_h2o__10m_c',   np.isnan(sos_ds['w_h2o__10m_c']).sum().item())
print('w_h2o__15m_c',   np.isnan(sos_ds['w_h2o__15m_c']).sum().item())
print('w_h2o__20m_c',   np.isnan(sos_ds['w_h2o__20m_c']).sum().item())
print('w_h2o__3m_ue',   np.isnan(sos_ds['w_h2o__3m_ue']).sum().item())
print('w_h2o__10m_ue',  np.isnan(sos_ds['w_h2o__10m_ue']).sum().item())
print('w_h2o__3m_uw',   np.isnan(sos_ds['w_h2o__3m_uw']).sum().item())
print('w_h2o__10m_uw',  np.isnan(sos_ds['w_h2o__10m_uw']).sum().item())
print('w_h2o__3m_d',    np.isnan(sos_ds['w_h2o__3m_d']).sum().item())
print('w_h2o__10m_d',   np.isnan(sos_ds['w_h2o__10m_d']).sum().item())

w_h2o__3m_c 7444
w_h2o__5m_c 8593
w_h2o__10m_c 6260
w_h2o__15m_c 10735
w_h2o__20m_c 11917
w_h2o__3m_ue 7767
w_h2o__10m_ue 9704
w_h2o__3m_uw 7403
w_h2o__10m_uw 10310
w_h2o__3m_d 9461
w_h2o__10m_d 8716


# Aggregate latent heat flux covariances into hourly

Using the correct (Reynolds averaging) method.

Switch to pandas dataframe because for greatly improved resampling performance.

In [14]:
sos_df = sos_ds.to_dataframe()

In [15]:
def resample_moments(df, covariance_variable_sets, new_frequency, n_in_new_re_length, skipna=True):
    """Combines moments into longer time periods, using reynolds averaging. Built to use with SOS 
    datasets. Resampling covariances which have been calculated for a specific Reynolds
    averaging length (e.g. the SOS datasets are averaged to 5minutes), you need both the mean
    values and covariance. For example, the variable `w_h2o__3m_c` is associated with mean values
    `w_3m_c` and `h2o_3m_c`. To reasmple `w_h2o__3m_c` to another averaging length, we need the three
    variables.

    Following the math outlined here:
    https://www.eol.ucar.edu/content/combining-short-term-moments-longer-time-periods

    Args:
        df (pd.DataFrame): dataframe containing the three columns required for the calculation. An 
        input dataframe could look like this:
                                    w_h2o__3m_c    w_3m_c  h2o_3m_c
            time                                                
            2022-11-04 00:02:30    -0.000873  0.015235  3.049837
            2022-11-04 00:07:30    -0.000564  0.056043  3.007558
            2022-11-04 00:12:30    -0.000953  0.060759  3.041484
            2022-11-04 00:17:30    -0.000649  0.026777  3.057822
            2022-11-04 00:22:30    -0.000559  0.013143  3.095043
        covariance_variable_sets (list[tuple]): A list of tuples, where each tuple contains 3 strings,
            the variables that correspond to, in order:
                - the covariance you are trying to recalculate for a longer time period
                - one component of that covariance
                - the other component of that covariance
            For example, the input for this parameter might look like this:
            [
                ('w_h2o__3m_c', 'w_3m_c', 'h2o_3m_c'),
                ('w_h2o__5m_c', 'w_5m_c', 'h2o_5m_c'),
                ('w_h2o__10m_c', 'w_10m_c', 'h2o_10m_c'),
            ]
        new_frequency (str): String interpretable by pandas/xarray that describes the reynolds length you 
            are resampling to. EG: '60Min'
        n_in_new_re_length (_type_): Number of 5 minute intervals that fit in the new_frequency. E.G. for
            new_frequency='60Min', you would provide 12.
        skipna (bool, optional): Whether to skip NaNs when calculating the new variables. Providing True
            will allow more moments to be calculated, but those moments may be inaccurate/non-representative.
            Providing False will result in more missing data.

    Returns:
        pd.DataFrame: Aggregated dataframe
    """
    agg_df_list = []
    for cov, w, h2o in covariance_variable_sets:
        agg_df = pd.DataFrame({
            cov: df.groupby(pd.Grouper(freq=new_frequency)).apply(
                lambda row: 
                    (1/n_in_new_re_length)*(row[cov] + row[w]*row[h2o]).sum(skipna=skipna)
                    - (
                        (1/n_in_new_re_length)*row[w].sum(skipna=skipna)
                        * (1/n_in_new_re_length)*row[h2o].sum(skipna=skipna)
                    )
            )
        })
        agg_df_list.append(agg_df)
    combined_agg_df = agg_df_list[0]
    for i in range(1, len(agg_df_list)):
        combined_agg_df = combined_agg_df.join(agg_df_list[i])
    return combined_agg_df

In [16]:
lhflux_covariance_variable_sets = [
    ('w_h2o__3m_c',     'w_3m_c',   'h2o_3m_c'),
    ('w_h2o__5m_c',     'w_5m_c',   'h2o_5m_c'),
    ('w_h2o__10m_c',    'w_10m_c',  'h2o_10m_c'),
    ('w_h2o__15m_c',    'w_15m_c',  'h2o_15m_c'),
    ('w_h2o__20m_c',    'w_20m_c',  'h2o_20m_c'),   
    ('w_h2o__3m_ue',    'w_3m_ue',  'h2o_3m_ue'),
    ('w_h2o__10m_ue',   'w_10m_ue', 'h2o_10m_ue'),
    ('w_h2o__3m_uw',    'w_3m_uw',  'h2o_3m_uw'),
    ('w_h2o__10m_uw',   'w_10m_uw', 'h2o_10m_uw'),
    ('w_h2o__3m_d',     'w_3m_d',   'h2o_3m_d'),
    ('w_h2o__10m_d',    'w_10m_d',  'h2o_10m_d'),
]
shflux_covariance_variable_sets = [
    ('w_tc__3m_c',     'w_3m_c',   'tc_3m_c'),
    ('w_tc__5m_c',     'w_5m_c',   'tc_5m_c'),
    ('w_tc__10m_c',    'w_10m_c',  'tc_10m_c'),
    ('w_tc__15m_c',    'w_15m_c',  'tc_15m_c'),
    ('w_tc__20m_c',    'w_20m_c',  'tc_20m_c'),   
    ('w_tc__3m_ue',    'w_3m_ue',  'tc_3m_ue'),
    ('w_tc__10m_ue',   'w_10m_ue', 'tc_10m_ue'),
    ('w_tc__3m_uw',    'w_3m_uw',  'tc_3m_uw'),
    ('w_tc__10m_uw',   'w_10m_uw', 'tc_10m_uw'),
    ('w_tc__3m_d',     'w_3m_d',   'tc_3m_d'),
    ('w_tc__10m_d',    'w_10m_d',  'tc_10m_d'),
]

lhflux_agg_10Min_df = resample_moments(sos_df, lhflux_covariance_variable_sets, '10Min', 2, skipna=True)
shflux_agg_10Min_df = resample_moments(sos_df, shflux_covariance_variable_sets, '10Min', 2, skipna=True)

lhflux_agg_30Min_df = resample_moments(sos_df, lhflux_covariance_variable_sets, '30Min', 6, skipna=True)
shflux_agg_30Min_df = resample_moments(sos_df, shflux_covariance_variable_sets, '30Min', 6, skipna=True)

lhflux_agg_60Min_df = resample_moments(sos_df, lhflux_covariance_variable_sets, '60Min', 12, skipna=True)
shflux_agg_60Min_df = resample_moments(sos_df, shflux_covariance_variable_sets, '60Min', 12, skipna=True)

lhflux_agg_120Min_df = resample_moments(sos_df, lhflux_covariance_variable_sets, '120Min', 24, skipna=True)
shflux_agg_120Min_df = resample_moments(sos_df, shflux_covariance_variable_sets, '120Min', 24, skipna=True)


lhflux_agg_5Min_df = sos_df[lhflux_agg_10Min_df.columns]
shflux_agg_5Min_df = sos_df[shflux_agg_10Min_df.columns]

In [19]:
lhflux_agg_30Min_df['w_h2o__3m_c'].loc['2022-11-30 00:00:00':'2023-05-09 23:30:00'].to_csv("hihihih.csv")

In [26]:
lhflux_agg_30Min_df['w_h2o__3m_c'].loc["20221130":"20230510"].sum()*60*30/1000

35.0041157195032

In [21]:
print(round(100*lhflux_agg_5Min_df['w_h2o__3m_c'].isna().sum()/len(lhflux_agg_5Min_df), 2))
print(round(100*lhflux_agg_10Min_df['w_h2o__3m_c'].isna().sum()/len(lhflux_agg_10Min_df), 2))
print(round(100*lhflux_agg_30Min_df['w_h2o__3m_c'].isna().sum()/len(lhflux_agg_30Min_df), 2))
print(round(100*lhflux_agg_60Min_df['w_h2o__3m_c'].isna().sum()/len(lhflux_agg_60Min_df), 2))
print(round(100*lhflux_agg_120Min_df['w_h2o__3m_c'].isna().sum()/len(lhflux_agg_120Min_df), 2))

11.34
0.0
0.0
0.0
0.0


In [ ]:
cumsub_src5 = lhflux_agg_5Min_df.loc['20221130': '20230509'].sum()*5*60/1000
cumsub_src5.name = '5min'

cumsub_src10 = lhflux_agg_10Min_df.loc['20221130': '20230509'].sum()*10*60/1000
cumsub_src10.name = '10min'

cumsub_src30 = lhflux_agg_30Min_df.loc['20221130': '20230509'].sum()*30*60/1000
cumsub_src30.name = '30min'

cumsub_src60 = lhflux_agg_60Min_df.loc['20221130': '20230509'].sum()*60*60/1000
cumsub_src60.name = '60min'

cumsub_src120 = lhflux_agg_120Min_df.loc['20221130': '20230509'].sum()*120*60/1000
cumsub_src120.name = '120min'

cumsub_comparison_df = pd.DataFrame(cumsub_src5).join(
    pd.DataFrame(cumsub_src10)
).join(
    pd.DataFrame(cumsub_src30)
).join(
    pd.DataFrame(cumsub_src60)
).join(
    pd.DataFrame(cumsub_src120)
).reset_index().rename(columns={'index':'variable'})
cumsub_comparison_df['tower'] = cumsub_comparison_df['variable'].apply(lambda s: s.split('_')[-1])
cumsub_comparison_df['height'] = cumsub_comparison_df['variable'].apply(lambda s: int(s.split('_')[-2].split('m')[0]))

In [ ]:
meanlhflux_src5 = lhflux_agg_5Min_df.loc['20221130': '20230509'].mean()
meanlhflux_src5.name = '5min'

meanlhflux_src10 = lhflux_agg_10Min_df.loc['20221130': '20230509'].mean()
meanlhflux_src10.name = '10min'

meanlhflux_src30 = lhflux_agg_30Min_df.loc['20221130': '20230509'].mean()
meanlhflux_src30.name = '30min'

meanlhflux_src60 = lhflux_agg_60Min_df.loc['20221130': '20230509'].mean()
meanlhflux_src60.name = '60min'

meanlhflux_src120 = lhflux_agg_120Min_df.loc['20221130': '20230509'].mean()
meanlhflux_src120.name = '120min'

meanlhflux_comparison_df = pd.DataFrame(meanlhflux_src5).join(
    pd.DataFrame(meanlhflux_src10)
).join(
    pd.DataFrame(meanlhflux_src30)
).join(
    pd.DataFrame(meanlhflux_src60)
).join(
    pd.DataFrame(meanlhflux_src120)
).reset_index().rename(columns={'index':'variable'})
meanlhflux_comparison_df['tower'] = meanlhflux_comparison_df['variable'].apply(lambda s: s.split('_')[-1])
meanlhflux_comparison_df['height'] = meanlhflux_comparison_df['variable'].apply(lambda s: int(s.split('_')[-2].split('m')[0]))

In [ ]:
meanshflux_src5 = shflux_agg_5Min_df.loc['20221130': '20230509'].mean()
meanshflux_src5.name = '5min'

meanshflux_src10 = shflux_agg_10Min_df.loc['20221130': '20230509'].mean()
meanshflux_src10.name = '10min'

meanshflux_src30 = shflux_agg_30Min_df.loc['20221130': '20230509'].mean()
meanshflux_src30.name = '30min'

meanshflux_src60 = shflux_agg_60Min_df.loc['20221130': '20230509'].mean()
meanshflux_src60.name = '60min'

meanshflux_src120 = shflux_agg_120Min_df.loc['20221130': '20230509'].mean()
meanshflux_src120.name = '120min'

meanshflux_comparison_df = pd.DataFrame(meanshflux_src5).join(
    pd.DataFrame(meanshflux_src10)
).join(
    pd.DataFrame(meanshflux_src30)
).join(
    pd.DataFrame(meanshflux_src60)
).join(
    pd.DataFrame(meanshflux_src120)
).reset_index().rename(columns={'index':'variable'})
meanshflux_comparison_df['tower'] = meanshflux_comparison_df['variable'].apply(lambda s: s.split('_')[-1])
meanshflux_comparison_df['height'] = meanshflux_comparison_df['variable'].apply(lambda s: int(s.split('_')[-2].split('m')[0]))

In [ ]:
(alt.Chart(cumsub_comparison_df).transform_fold(
    ['5min', '10min', '30min', '60min', '120min']
).mark_point().encode(
    alt.X("key:O").sort(['5min', '10min', '30min', '60min', '120min']).title(None).axis(None),
    alt.Y("value:Q").title("Total sublimation (mm)"),
    alt.Color("tower:N"),
    alt.Facet("height:O"),
).properties(height = 100) &\
alt.Chart(meanlhflux_comparison_df).transform_fold(
    ['5min', '10min', '30min', '60min', '120min']
).mark_point().encode(
    alt.X("key:O").sort(['5min', '10min', '30min', '60min', '120min']).title(None).axis(None),
    alt.Y("value:Q").title("<w'q'> (g/m^2/s)"),
    alt.Color("tower:N"),
    alt.Facet("height:O").title(None).header(labelFontSize=0),
).properties(height = 100) &\
alt.Chart(meanshflux_comparison_df).transform_fold(
    ['5min', '10min', '30min', '60min', '120min']
).mark_point().encode(
    alt.X("key:O").sort(['5min', '10min', '30min', '60min', '120min']).title("Averaging length"),
    alt.Y("value:Q").title("<w'T'> (˚C m/s)"),
    alt.Color("tower:N"),
    alt.Facet("height:O").title(None).header(labelFontSize=0),
).properties(height = 100)).display(renderer='svg')

## Incorrect method (taking the mean)

In [ ]:
wrong_combined_agg_df = sos_df[[
    'w_h2o__3m_c',
    'w_h2o__5m_c',
    'w_h2o__10m_c',
    'w_h2o__15m_c',
    'w_h2o__20m_c',
]].groupby(pd.Grouper(freq='60Min')).mean()

In [ ]:
wrong_combined_agg_df.loc['20221221': '20221224'].reset_index().melt(id_vars='time')

In [ ]:
import altair as alt
(
    alt.Chart(
        wrong_combined_agg_df.loc['20221221': '20221224'].reset_index().melt(id_vars='time')
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").title('Latent heat flux (g/m^2/s)'),
        alt.Color("variable:O").scale(scheme='rainbow', domain=[
                'w_h2o__3m_c',
                'w_h2o__5m_c',
                'w_h2o__10m_c',
                'w_h2o__15m_c',
                'w_h2o__20m_c',
        ])
    ).properties(title = 'Incorrect covariance averages') | alt.Chart(
        combined_agg_df.loc['20221221': '20221224'].reset_index().melt(id_vars='time')
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").title('Latent heat flux (g/m^2/s)'),
        alt.Color("variable:O").scale(scheme='rainbow', domain=[
                'w_h2o__3m_c',
                'w_h2o__5m_c',
                'w_h2o__10m_c',
                'w_h2o__15m_c',
                'w_h2o__20m_c',
        ])
    ).properties(title = 'Correct covariance averages')
).resolve_scale(x='shared', y='shared', color='shared').display(renderer='svg')

Test with a single variable using simpler code

In [ ]:
new_values = []
for i in range(0, int(len(sos_ds.time)/12)):
    start_i = i*12
    end_i = (i+1)*12
    cov_values = sos_ds['w_h2o__3m_c'][start_i: end_i].values
    w_values = sos_ds['w_3m_c'][start_i: end_i].values
    h2o_values = sos_ds['h2o_3m_c'][start_i: end_i].values
    new_val  = (
        (1/12)*(cov_values + w_values*h2o_values).sum() 
        - (
            (1/12)*w_values.sum()
            * (1/12)*h2o_values.sum()
        )
    )
    new_values.append(new_val)

In [ ]:
np.unique(combined_agg_df['w_h2o__3m_c'].values - np.array(new_values))

# Save 60 min lh flux values

In [ ]:
combined_agg_df.to_xarray().to_netcdf('sos_hourly_aggregated_lhflux_ds.nc')

# Check that the cumulative sublimation measurements look right.

In [ ]:
from metpy.constants import density_water

In [ ]:
sos_ds_mm = sos_ds*5*60/density_water.magnitude

In [ ]:
sos_ds_mm['w_h2o__3m_c'].cumsum().plot()
sos_ds_mm['w_h2o__5m_c'].cumsum().plot()
sos_ds_mm['w_h2o__10m_c'].cumsum().plot()
sos_ds_mm['w_h2o__15m_c'].cumsum().plot()
sos_ds_mm['w_h2o__20m_c'].cumsum().plot()